imports & connection

In [5]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import sqlite3

DWengine = create_engine("mssql+pyodbc://LEGION\\SQLEXPRESS/Data Warehouse?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

Create dataframes from DB tables

In [8]:
course_df = pd.read_sql_query('SELECT * FROM course', DWengine)
order_df = pd.read_sql_query('SELECT * FROM [order]', DWengine)
order_details_df = pd.read_sql_query('SELECT * FROM order_details', DWengine)
order_method_df = pd.read_sql_query('SELECT * FROM order_method', DWengine)
retailer_site_df = pd.read_sql_query('SELECT * FROM retailer_site', DWengine)
return_reason_df = pd.read_sql_query('SELECT * FROM return_reason', DWengine)
returned_item_df = pd.read_sql_query('SELECT * FROM returned_item', DWengine)
sales_staff_df = pd.read_sql_query('SELECT * FROM sales_Staff', DWengine)
training_df = pd.read_sql_query('SELECT * FROM training', DWengine)

Source Data Model connection

In [ ]:
SDMengine = create_engine("mssql+pyodbc://LEGION\\SQLEXPRESS/sdm?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,SALES_STAFF_CODE,SALES_BRANCH_CODE,ORDER_DATE,ORDER_METHOD_CODE
